In [1]:
import sys
import time
from pathlib import Path
import ssl

sys.path.append(str(Path('../').resolve()))
assert 'python-nostr' in [Path(path).name for path in sys.path]

from nostr.key import PrivateKey, PublicKey
from client import ConnectedClient

## make a helper method to get a private key from disk
or allow user input

In [2]:
def get_private_key() -> str:
    key_file = Path('./private_key.txt') # saving a private key for testing

    if not key_file.exists():
        pk_hex = input()
        if len(pk_hex.strip()) == 0:
            pk_hex = PrivateKey().hex()
            print(f'generated new private key: {pk_hex}')
        else:
            print('using user private key')
    else:
        with open(key_file, 'r') as f:
            pk_hex = f.read()
            # private_key = PrivateKey(bytes.fromhex(hex))
            private_key = PrivateKey.from_hex(pk_hex)
        public_key = private_key.public_key
        print(f'Loaded private key from {key_file}')
        print(f'Public key: {public_key.bech32()}')
    return pk_hex

# client
instantiating a basic client and testing connections

In [3]:
pk_hex = get_private_key()

with ConnectedClient(ssl_options={'cert_reqs': ssl.CERT_NONE},
                     private_key_hex=pk_hex) as connected_client:
    print(f'loaded public key is {connected_client.public_key.bech32()}\n')
    for relay in connected_client.relay_manager:
        assert relay.ws.sock is not None
        print(f'connection to {relay.url} is alive!')

for relay in connected_client.relay_manager:
    assert relay.ws.sock is None
    print(f'connection to {relay.url} is dead.')

Loaded private key from private_key.txt
Public key: npub19nqc7plddvevd8sgfpahzuput0q9hwrfhk8crj9k73fpg8qwjx3sk6224n
loaded public key is npub19nqc7plddvevd8sgfpahzuput0q9hwrfhk8crj9k73fpg8qwjx3sk6224n

connection to wss://nostr-2.zebedee.cloud is alive!
connection to wss://nostr-2.zebedee.cloud is dead.


# make a simple text input client

we will instantiate it with the proper options first

In [5]:
from client import TextInputClient
import ssl

pk_hex = get_private_key()

text_client = TextInputClient(ssl_options={'cert_reqs': ssl.CERT_NONE}, private_key_hex=pk_hex, allow_duplicates=False)

Loaded private key from private_key.txt
Public key: npub19nqc7plddvevd8sgfpahzuput0q9hwrfhk8crj9k73fpg8qwjx3sk6224n


## run it
with a simple `with` context. the `__enter__` method calls the `run` command of the client so we can just call `pass` so that nothing else happens after we exit the `run` method before the we `__exit__` the clients and the connections are closed.

In [6]:
with text_client:
    pass

select a command:
                	0	E(x)it
                	1	publish note
                	2	get last 10 notes by you
                	3	get last 10 from hex of author
                	4	delete an event
                	5	check deletions
                	6	get metadata by hex of user
                
author: 2cc18f07ed6b32c69e08487b71703c5bc05bb869bd8f81c8b6f452141c0e91a3
event id: 27cf065883f63084af15bdfdab9e24d68126ca0ec1522675b47c05781e005888
	this is but a test
author: 2cc18f07ed6b32c69e08487b71703c5bc05bb869bd8f81c8b6f452141c0e91a3
event id: b6b21ee8f696998d5f4f0a62cde449edab46353ba8c91ceeb5c9a15a3b895b85
	please ignore this test
author: 2cc18f07ed6b32c69e08487b71703c5bc05bb869bd8f81c8b6f452141c0e91a3
event id: 4d1c3b10f2e1f7a5fd655d323f2b4cade53284b9765a0ef15cdce6aec0b6fc88
	test!
author: 2cc18f07ed6b32c69e08487b71703c5bc05bb869bd8f81c8b6f452141c0e91a3
event id: 260012d4b5ca660bca4f945d5c2cba0292c1a3f4a503244ba7a98a745cadd3cb
	testing again
author: 2cc18f07ed6b32c69e08487b71703

In [8]:
test = text_client.message_store.get('7a1d36ce5dec9ff7bb1771d61bf6f71895400ce57f0919313a8dfc87772dda1c')

In [10]:
test.url

'wss://nostr-2.zebedee.cloud'